# Financial Data Reporting 

## --- Steps to Create DataFrame in Python and Connect to SQL --- 
### Step1. Data Preparation (Exception Handling function)
### Step2. Check if file exists
### Step3. Load Data
### Step4. Data Cleaning & Create Table 
### Step5. Move "Latest" File to Archived Folder
### Step6. Import  to SQL
## -------------------------------------------------------------------------------------------

### Step1. Data Preparation --------------------------------------------------------------------

#### Step1.1 Import all libraries needed

In [17]:
import pandas as pd
import numpy as np
import os
from os import path
from pathlib import Path
import sys
from os import listdir
from os.path import isfile, join
import pyodbc
import string
from itertools import groupby
import os.path, time
import glob
import datetime

print("Step1.1 Complete!\n")

Step1.1 Complete!



#### Step1.2 Exception Handling function

In [18]:
def InsertErrorMessageInSQL (ErrorLocation, ErrorMessage):
    """ Insert Error Message/ Successful Message to SQL Corporate_KPI"""
    ErrorList = ['Python', None, ErrorLocation, ErrorMessage] # Template message to be inserted to Translog table
    
    # Insert data into SQL EBITDA table 
    try:
        cursor.execute("INSERT INTO translog (type, module_name, procedure_name, trans_details) VALUES (?,?,?,?)", ErrorList)
    
    # If exception occurs, all data transaction will be rolled back and all data erased.
    except Exception as e:
        cursor.rollback()
        print(e)
        print('transaction rolled back')

    else:
        cursor.commit()
        cursor.close() # keep the currsor active, otherwise close
    
    finally:
        print('Error Message Successfully inserted')
        conn.close() # Close Connection
        exit()
        
    return


In [19]:
# This is how to get comments of functions (good to look up what the function does)
print(InsertErrorMessageInSQL.__doc__)

 Insert Error Message/ Successful Message to SQL Corporate_KPI


In [20]:
# Connect to SQL Server 
# Reference: https://www.youtube.com/watch?v=LZPyuLnZI34

    
try:
    conn = pyodbc.connect(#"DSN=ConnectPythonWithSQL;"
                          "Driver={SQL Server Native Client 11.0};"
                          "Server=sqlserver4;"
                          "Database=Corporate_KPI;"
                          "Trusted_Connection=yes;")

except Exception as e:
    print(e)
    print("task is terminated")
    print("This error will not be inserted into SQL translog because it failed to connect to SQL to start with")
    exit()

else:
    # connect to cursor 
    cursor = conn.cursor()

### Step2. Check if file exists in folder --------------------------------------------------------------------

In [21]:
def create_file_list(sourcePath):
    """Creates list according to the files in the folder"""
    # Get all files in the folder 'Finance'
    fileNameList = [f for f in listdir(sourcePath) if isfile(join(sourcePath, f))]
    # Removing all non excel (.xlsx or .xlsm) files from the list
    fileNameList = [x for x in fileNameList if extensionFile in x or extensionFileMacro in x]
    
    return fileNameList

In [22]:
def check_file(list_files):
    """Checks if Excel file exists in the Designated Folder. Exit the system if no file found"""
    
    #If list_file is [](empty: no file in folder), then == False
    if bool(list_files) == False: 
        print("system will exit because no file detected")
        exit()
        
    return

### Step3. Load Data --------------------------------------------------------------------

In [23]:
def get_latest_file (sourcePath):
    """Get the latest uploaded file in the designated folder by the latest 'Data Modified Time' """
    
    file_type = '\*xlsx'
    file_typeMacro = '\*xlsm'
    #retrieve all files with '\*xlsx' and '\*xlsm' wiith glob funcation
    files = glob.glob(sourcePath + file_type)
    files.extend(glob.glob(sourcePath + file_typeMacro)) #files.extend(list_files)
    # get time of last modificatoin of the specific path and choose the most latest file 
    latest_file = max(files, key = os.path.getmtime) 
    file_name = os.path.basename(latest_file) # get the file name of the latest file 
    
    return latest_file, file_name 

In [24]:
try:
    sourcePath = r"\\nas1\users\shared files\Database Export-Import files\Financial"
    extensionFile = r".xlsx"
    extensionFileMacro = r".xlsm"


    #Create list of files in the folder
    fileNameList = create_file_list(sourcePath)
    print(fileNameList)

    #if there are no file, the system will end 
    check_file(fileNameList)

    #load data, if there is one file in the folder
    if len(fileNameList) == 1: 
        # For accessing only one "first" file
        df = pd.read_excel(sourcePath + r"/" +fileNameList[0], sheet_name = "Consolidated- FORECAST", header = None, index_col = None, na_values = ['None'])
        print(fileNameList[0], "is loaded!") 

    #Load "Latest" File date, if there are multiple file in the folder 
    elif len(fileNameList) > 1:

        # For accessing only one "latest" file and load
        ouput = get_latest_file (sourcePath)
        latest_file_result = list(ouput) # converting into list data type. Within the list, there are latest_file, file_name of the latest file
        df = pd.read_excel(latest_file_result[0], sheet_name = "Consolidated- FORECAST", header = None, index_col=None, na_values=['None'])
        print(latest_file_result[1], "is loaded!")

        # All files except "latest" are moved to "Archived" Folder 
        for n_files in range(0, len(fileNameList)):
            # if name of each file does not match the latest file name, then take the file to the archived folder
            if fileNameList[n_files] != latest_file_result[1]:
                os.rename(sourcePath + r"/" +fileNameList[n_files], r"\\nas1\users\shared files\Database Export-Import files\Financial\Archive" +  r"/" +fileNameList[n_files][:-5]  + str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")) + fileNameList[n_files][-5:])        
        print("All files except 'latest' are moved to 'Archived' Folder ")
        

except Exception as e: 
    print(e)
    InsertErrorMessageInSQL("Step2/Step3, Load Data", repr(e))
    
print("Step3 Complete!\n")

['11 Dec - Consolidated Forecast- notional update MACRO ROLL FORWARD2022-01-28_09_002022-01-28_15_00.xlsm']
11 Dec - Consolidated Forecast- notional update MACRO ROLL FORWARD2022-01-28_09_002022-01-28_15_00.xlsm is loaded!
Step3 Complete!



### Step4. Data Cleaning & Create Table --------------------------------------------------------------------

In [25]:
#TO DO: check for .values / why transpose?/ astype(int), argmax()

# Find last "Feburary". This is for getting last Feb until now to get the data for a current year." 
def find_last_Feburary (df):
    """ Find last "Feburary" column number """
    
    #Create detaframe of 'row5 and column 4 until the end', 'no NA value', 'remove old index in a column','Transposed'
    df = pd.DataFrame(df.iloc[5,4:].values).dropna(how='all').reset_index(drop = True).T 
    Feb_col = 0
    # reverse for loop to find last feb 
    # Loops to find coloum that is 'Feburary'
    for col_num in range(len(df.columns)-1, 0, -1):
        if df.iloc[0,col_num] == "Feb":
            Feb_col = col_num
            break
            
    return Feb_col

In [26]:
# Find forecasts column
def getFirstForecast(df):
    """Get the first month forecast"""
    
    #sort out data frame
    df2 = pd.DataFrame(df.iloc[4,:].values).dropna(how='all').reset_index(drop = True)
    # Get column number of 'Faroecast' 
    # change data type at the end 
    firstForecast = (df2.values == 'Forecast').argmax().astype(int)
    
    return firstForecast

## Step 4.1 - Get the data in a format we that we want.
### Step 4.1.1: Remove duplicate columns if a month is repeating consecutively.
### Step 4.1.2: Use 6th row as column names.
### Step 4.1.3: Get rows of data that contain terminal rows.
### Step 4.1.4: Get columns of data which contain actual and forecast data.
### Step 4.1.5: If the forecast month is Feb, then return last year's Jan + Feb, if not, return data from start of fiscal year until that month.
### Step 4.1.6: Add BU_ID based on first column and terminal names . 
### Step 4.1.7: Drop Lynnterm 6000 because as per requirements, we do not need it. 
### Step 4.1.8: Get fiscal_year from YTDbudget column that is in the format of FYxx, change it to 20xx and add it as a new column.
### Step 4.1.9: Add YTDBudget by searching 6th row for budget.



### schema at the end of this step: [Month1, Month2, ..., MonthX, YTDBudget, BU_id, Fiscal_year]


In [27]:
def getDesiredRowsAndColumns(df):
    """Add months, BU and budget as the column name.
       Months and Budget are found through search in fifth row. BU is the first column."""
        
    # Remove all columns with duplicated month name that is next to each other (preventing the case when Aylene puts two
    #Feburary" for forecasts because we will need to identify current year based on "Feburary")
    # Used transpose function for dataframe here because combining dataset later and need data to be on column position
    
    # Copied data frame and Transpose and rename 5 to 'month' on row (axis = 1 is column, 0 is row) df will be original data frame
    df_1 = df.copy().T.rename({5:"months"}, axis = 1)
    month_name_columns = df.T.iloc[:, 5] # getting transposed data frame from column of 5
    # drop consequtive duplicates and convert data to data frame type, and rename column 5 to month
    no_duplicated_columns = month_name_columns.loc[month_name_columns.shift() != month_name_columns].to_frame().rename({5:"months"}, axis = 1)
    removedDuplicated = df_1[df_1.months.isin(no_duplicated_columns.months)].T # filtering joins to combine two data sets and transpose to put everything back 
    df = removedDuplicated.copy()    

    df.columns = df.iloc[5,:].to_list()
    monthNames = ["Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","Jan"]
    firstTerminalRow = df[df.iloc[:,0] == "Terminals and Stevedoring"].index.values.astype(int)[0]+1
    lastTerminalRow = df[df.iloc[:,0] == "Intermodal"].index.values.astype(int)[0]-1
    allMonthsDF = df.iloc[firstTerminalRow:lastTerminalRow,df.columns.isin(monthNames)].dropna(how='all').reset_index(drop = True)

    # get latest forecast month name. This will be one of return values.
    indexOfFirstForecastMonth = monthNames.index(allMonthsDF.iloc[:,getFirstForecast(df)].name)
    ForecastMonth = allMonthsDF.keys().values[indexOfFirstForecastMonth]
    
    # if Forecasts = Feb, which means actual = Jan and forecast = feb. 
    if find_last_Feburary(df) == getFirstForecast(df):
        ActualandForcast = allMonthsDF.iloc[:,find_last_Feburary(df)-1:getFirstForecast(df)+1] # Aqcuire Jan on previous year and Feb
  
    else:
        ActualandForcast = allMonthsDF.iloc[:,find_last_Feburary(df):getFirstForecast(df)+1] # Aquire all month from feb this year 
        
    #here is when BU and YTD budget are added
    BU_idPlusYTDBudget = pd.DataFrame(data = {"BU_id":df.iloc[firstTerminalRow:lastTerminalRow,0],"YTDBudget":df.loc[firstTerminalRow:lastTerminalRow,"Budget"]}).dropna(how='all').reset_index(drop = True)

    #drop last row because it is redundant
    BU_idPlusYTDBudget = BU_idPlusYTDBudget.head(-1)
    df2 = ActualandForcast.copy()
    df2["BU_id"] = BU_idPlusYTDBudget.BU_id
    df2["YTDBudget"] = BU_idPlusYTDBudget.YTDBudget

    
    #Drop Lynnterm 6000
    df2 = df2.drop(df2[df2.BU_id == "Lynnterm 6000 account"].index.values[0],axis = 0).reset_index(drop = True)  
    
    #add fiscal year. This will be another return value.
    fiscalYearHavingFY = df.iloc[3,df.iloc[5,:].values == 'Budget'].item()
    FiscalYearEditted1 = fiscalYearHavingFY.replace('F', '2')
    fiscalYear = FiscalYearEditted1.replace('Y', '0')
    df2["Fiscal_year"] = int(fiscalYear)
    

    
    #return df2 because
        #it is basically what we are reading from excel but we made some changes to format. This is main output of this function.
    #return fiscalYear because
        #in step 4.3.3 we want to get current fiscal year data from SQL server    
    #return ForecastMonth because:
        #in step 4.3.3 we want to make a differentiation between forecast and actual data columns and months.
    #return ActualandForcast because
        #in step 4.5 we want to limit months in SQL dataframe, to those which have new data in present months in excel sheet. 
    return df2,fiscalYear,ForecastMonth,ActualandForcast

In [28]:
try:
    trimmedDF,fiscalYear,ForecastMonth,onlyActualandForecast = getDesiredRowsAndColumns(df)
except Exception as e: 
    print(e)
    InsertErrorMessageInSQL("Step 4.1. Data Cleaning: Get the data in a format we that we want",repr(e))

    
print("Step4.1 Complete!\n")


Step4.1 Complete!



## Step4.2: From previous step, expand the month columns so we will not have column months anymore. Rather, we will have multiple rows for each month.

In [29]:
def getExpandedMonths(trimmedDF):
    expandMonths = trimmedDF.melt(id_vars = ['Fiscal_year','BU_id','YTDBudget'], var_name='Fiscal_month', value_name='Actual')
    expandMonths = expandMonths.reindex(columns = expandMonths.columns.tolist() + ["Forecasts"])

    #read as new column in expandMonths, known as Forecasts equals ..
    expandMonths['Forecasts'].where(~(expandMonths.Fiscal_month == ForecastMonth), other= expandMonths['Actual'], inplace=True)
    expandMonths['Actual'].where(~(expandMonths.Fiscal_month == ForecastMonth), other= None, inplace=True)
    expandMonths['YTDBudget'].where(~(expandMonths.Fiscal_month != ForecastMonth), other= None, inplace=True)
    #change the position of columns
    new_cols = ['Fiscal_year','Fiscal_month','BU_id','Actual','Forecasts','YTDBudget']
    expandMonths=expandMonths.reindex(columns=new_cols)
    return expandMonths

In [30]:
try:
    expandMonths = getExpandedMonths(trimmedDF)
    
except Exception as e: 
    print(e)
    InsertErrorMessageInSQL("Step 4.2. Data Cleaning: From previous step, expand the month columns to have multiple rows for each month.",repr(e))


print("Step4.2 Complete!\n")
    

Step4.2 Complete!



## From this step onwards, we start a process of getting the current year's data from SQL. The reason we do that is because in order to calculate monthly budget, we need to get sum of up to last month's budget from SQL and then decrease it from ytdBudget we currently have. So that is how we are going to do it.

## Step4.3: Connect to SQL server, get EBITDA and BU tables then join them to get BU names on EBITDA instead of id. After that, limit the data to first month of current fiscal year up to current month, and get sum of budgets for each BU.

### Step 4.3.1: Connect to SQL server and get data 
### Step 4.3.2: Join the tables received from SQL to get BU names instead of BU_ID. This is used becasue in editted excel data, also known as TrimmedDF2, we have BU names and later we want to join this dataframe with TrimmedDF.
### Step 4.3.3: Limit the data to read from SQL: Get the data from SQL only for when the fiscal year equals the fiscal year in excel sheet. Then get the data from start of fiscal year until the previous ACTUAL month(no forecast) . We are going to calculate sum of budgets by BU based sum of previous months from current fiscal year. 
### Step 4.3.4: get a number as sum of budgets for each BU based on SQLcurrentYearandYTDMonths. So we will calculate sum of budgets based on each terminal for the CURRENT FISCAL YEAR and MONTHS that we have actual data from SQL.

### schema at the end of this step: [BU, Dollar amount]

In [36]:
def getSQLDataYTDBudgetGroupByBU(onlyActualandForecast,ForecastMonth,fiscalYear):
    
    #Get data of EBITDA table from SQL
    results = cursor.execute("SELECT * FROM dbo.EBITDA").fetchall()
    # Convert pyodbc.Row to dict (data types)
    dictionary_sqlEBITDA = []
    sqlEBITDA_columnNames = [column[0] for column in cursor.description]
    for record in results:
        dictionary_sqlEBITDA.append(dict(zip(sqlEBITDA_columnNames, record)))
    sqlEBITDA = pd.DataFrame(dictionary_sqlEBITDA)  
    
    #Get data of BU table from SQL
    results = cursor.execute("SELECT * FROM dbo.BU").fetchall()
    # Convert pyodbc.Row to dict (data types)
    dictionary_BU = []
    BU_columnNames = [column[0] for column in cursor.description]
    for record in results:
        dictionary_BU.append(dict(zip(BU_columnNames, record)))
    sqlBU = pd.DataFrame(dictionary_BU)
    
    #Step 4.3.2: Join the tables received from SQL to get BU names instead of BU_ID. 
    #This is used becasue in editted excel data, also known as TrimmedDF2, we have BU names and later we want to 
    #join this dataframe with TrimmedDF.
    joinsqlEBITDAandBU = pd.merge(sqlEBITDA, sqlBU, left_on = 'BU_id', right_on = 'id').drop(['id_x','BU_id','Desc','id_y'],axis=1)
    
    # Step 4.3.3: Limit the data to read from SQL: Get the data from SQL only for when the fiscal year equals the 
    #fiscal year in excel sheet. Then get the data from start of fiscal year until the previous ACTUAL month(no forecast). 
    #We are going to calculate sum of budgets by BU based sum of previous months from current fiscal year. 
    SQLcurrentYear = joinsqlEBITDAandBU[joinsqlEBITDAandBU['Fiscal_year']==fiscalYear]
    allMonths = onlyActualandForecast.columns.values
    allMonthsExceptForcast = allMonths[allMonths != ForecastMonth]
    SQLcurrentYearandYTDMonths = SQLcurrentYear[SQLcurrentYear.Fiscal_month.isin(allMonthsExceptForcast)]
    
    #### Step 4.3.4: get a number as sum of budgets for each BU based on SQLcurrentYearandYTDMonths. So we will calculate
    #sum of budgets based on each terminal for the CURRENT FISCAL YEAR and MONTHS that we have actual data from SQL.
    #join on id to get BU Name instead of BU ids in sqlEBITDA. 
    #In joined dataframe, the new column is called BU and not BU_id anymore    
    SQLBudgetGroupByBU = SQLcurrentYearandYTDMonths.groupby(by='BU_Financial').sum().Budgets
    SQLBudgetGroupByBU.name = "SumofPreviousMonthBudgets"
    return SQLBudgetGroupByBU,allMonthsExceptForcast

In [38]:
try:
    SQLBudgetGroupByBU,allMonthsExceptForcast = getSQLDataYTDBudgetGroupByBU(onlyActualandForecast,ForecastMonth,fiscalYear)
except Exception as e: 
    print(e)
    InsertErrorMessageInSQL("Step4.3. Data Cleaning: Connect to SQL server, get EBITDA and BU tables then get sum of budget by BU.",repr(e))

print("Step4.3 Complete!\n")
SQLBudgetGroupByBU

too many values to unpack (expected 2)
('22001', '[22001] [Microsoft][SQL Server Native Client 11.0][SQL Server]String or binary data would be truncated. (8152) (SQLExecDirectW); [22001] [Microsoft][SQL Server Native Client 11.0][SQL Server]The statement has been terminated. (3621)')
transaction rolled back
Error Message Successfully inserted
Step4.3 Complete!



BU_Financial
BCVPC                          -153771.252
CVS/ PNWTS                     -889945.053
G3- Post construction fee      1833333.333
LogCo                           551538.462
Lynnterm EBITDA                3628464.923
Lynnterm Operational EBITDA    3628464.923
Other                           979168.054
Ship Ops- Island               1994062.000
Ship Ops- Mainland             4183880.902
Squamish                       1424855.077
Tidal Harmony/Cowbay            119538.462
Univar                          646153.846
Victoria/ Ogden Point          -888645.743
Name: SumofPreviousMonthBudgets, dtype: object

## Step 4.4: We know in expandMonths, only the latest month contains YTD budget, one budget for each BU. Now we want to change that YTD budget to current month's based on calculation in last step, SQLBudgetGroupByBU. In last step we got budget from start of current fiscal year to previous month. Therefore, if we calculate YTDBudget in expandMonths - BU Budgets in SQLBudgetGroupByBU, we get the monthly budget.

## Step 4.4.1: Filter expandMonths from 4.2 data to current year and months. Differentiate forecast data from actual data.  
## Step 4.4.2: If forecast month is Feb then just return forecast data from previous step. That's because in this case ytdBudget = Budget. Note: if the forecast month is february, we are going to have problems with fiscal year because fiscal year is now x+1 and we have forecast for february x+1 but we also have actual data for January x so we found a way to handle that by deducing the January year - 1.
## Step 4.4.3: Else, inner join forecast data with SQLBudgetGroupByBU from 4.4 by BU. Then calculate budget = excel dataframe YTDbudget - BUbudget (sum of all previous months.)  

## Step 4.4.4: Now append the actual data and forecast data with budget per month, and this is our final dataframe. 
## Step 4.4.5: Finally, fix the formatting of years and change NA values to None because it might get problematic in SQL server. 


In [17]:
def calculateMonthlyBudget(expandMonths, SQLBudgetGroupByBU,allMonthsExceptForcast):
    #get current month and year data in expandMonths. It contains all the values we want plus 
    #YTD budget and we want to change that to monthly budget.
    currentYearDataInExpandMonths = expandMonths[expandMonths['Fiscal_year']==int(fiscalYear)]
    forecastDataInExpandMonths = currentYearDataInExpandMonths[~currentYearDataInExpandMonths.Fiscal_month.isin(allMonthsExceptForcast)]
    
    actualDataInExpandMonths = currentYearDataInExpandMonths[currentYearDataInExpandMonths.Fiscal_month.isin(allMonthsExceptForcast)]
    actualDataInExpandMonths = actualDataInExpandMonths.drop('YTDBudget',axis = 1)


    #Join with SQLBudgetGroupByBU to get until last month's YTD budget
    #Read if the forecast month is february...
    if find_last_Feburary(df) == getFirstForecast(df):
        print("Feburary value is Forecast")
        #Here just get February data from expandMonths and do not do any joins with SQL because actually here YTDBudget = Budget in SQL
        joinedSQLandExcelForecastForBudget = expandMonths[expandMonths['Fiscal_month']== 'Feb'].rename(columns = {'YTDBudget':'Budgets'})
        actualDataInExpandMonths.Fiscal_year = actualDataInExpandMonths.Fiscal_year - 1 

    else:
        print("Feburary value is Actual")
        joinedSQLandExcelForecastForBudget = pd.merge(forecastDataInExpandMonths, SQLBudgetGroupByBU, left_on = 'BU_id', right_on = 'BU_Financial')
        #Formula
        joinedSQLandExcelForecastForBudget['Budgets'] = joinedSQLandExcelForecastForBudget['YTDBudget'] - (joinedSQLandExcelForecastForBudget['SumofPreviousMonthBudgets']).astype(float)
        joinedSQLandExcelForecastForBudget=joinedSQLandExcelForecastForBudget.drop(["SumofPreviousMonthBudgets","YTDBudget"],axis = 1)


    #Append with rest of data
    result = actualDataInExpandMonths.append(joinedSQLandExcelForecastForBudget)

    #Fix the formating and None values
    result["Fiscal_year"] = result["Fiscal_year"].astype(int)
    result["Fiscal_month"] = result["Fiscal_month"].astype(str)
    result["Actual"] = result["Actual"].astype(float)
    result["Forecasts"] = result["Forecasts"].astype(float)
    result["Budgets"] = result["Budgets"].astype(float)

    result["Actual"] = result["Actual"].replace({np.nan:None})
    result["Forecasts"] = result["Forecasts"].replace({np.nan:None})
    result["Budgets"] = result["Budgets"].replace({np.nan:None})
    return result
    


In [18]:
try:
    result = calculateMonthlyBudget(expandMonths, SQLBudgetGroupByBU,allMonthsExceptForcast)
    final_table = result
except Exception as e: 
    print(e)
    InsertErrorMessageInSQL("Step 4.4: Data Cleaning: we calculate YTDBudget in expandMonths - BU Budgets in SQLBudgetGroupByBU, we get the monthly budget.",repr(e))

print("Step4.4 Complete!\n")

Feburary value is Actual
Step4.4 Complete!



### Step5. Move "Latest" File to Archived Folder --------------------------------------------------------------------


In [19]:
# Acquire file in the Designated Folder (file that are left in folder)
try:
    fileNameList_Latest = create_file_list(sourcePath) # at this point, there will be only one file existed in this list (in the folder) if tehre are multiple files, then moved in Step 3
    # When transfering the file into Archived folder, by adding date and time on file name avoide having multiple exact same file on Archived folder. Otherwise error would occur.
    os.rename(sourcePath + r"/" +fileNameList_Latest[0] , r"\\nas1\users\shared files\Database Export-Import files\Financial\Archive" + r"/" +fileNameList_Latest[0][:-5]  + str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")) + fileNameList_Latest[0][-5:])   

except Exception as e: 
    print(e)
    InsertErrorMessageInSQL("Step5. Move'Latest' File to Archived Folder",repr(e))
    print("Step5 Complete!\n")

print("Step5 Complete!\n")


Step5 Complete!



### Step.6 Import to SQL --------------------------------------------------------------------

In [20]:
final_table

,Fiscal_year,Fiscal_month,BU_id,Actual,Forecasts,Budgets
0,2022,Feb,Ship Ops- Mainland,171947.39,None,None
1,2022,Feb,Ship Ops- Island,170609.0,None,None
2,2022,Feb,Univar,50287.6,None,None
3,2022,Feb,Other,97539.01,None,None
4,2022,Feb,Victoria/ Ogden Point,-97571.44,None,None
...,...,...,...,...,...,...
8,2022,Dec,Squamish,None,23014.0,148422.403923
9,2022,Dec,LogCo,None,60000.0,57451.923538
10,2022,Dec,G3- Post construction fee,None,166666.0,166666.666333
11,2022,Dec,Lynnterm Operational EBITDA,None,927945.0,None


#### Step6.1 Aqcure BU table & Convert BU_id

In [21]:
#Get data of bu table from SQL and replace BU name to BU ID 
try
    #Get data of bu table from SQL
    resultss = cursor.execute("SELECT * FROM dbo.bu").fetchall()

    # Convert pyodbc.Row to dict (data types)
    # build a list of column names and zip with each row to produce a list of dictionaries
    insertObject = [] # create empty list to append lists of data later
    # Get columns's names from 'resultss'
    columnNames = [column[0] for column in cursor.description]
    for record in resultss:
        insertObject.append(dict(zip(columnNames, record))) # zip with each row and column to produce a list of dictionaries

    # Replace BU name with BU id using bu table from SQL ( final table = table we have been working on here, insertObject = bu table from SQL)
    for num_BU in range(0, len(final_table["BU_id"].tolist())): # convert into list data type to COUNT BU id in 'final table'
        for num_BU_Table in range (0, len(insertObject)): # this for loop counts number of lists of insertObject
            if insertObject[num_BU_Table]["BU_Financial"] == final_table.iloc[num_BU,2]: # loop to find data in BU-Financial column from BU table in SQL matches with data in BU name in final table 
                # replace BU name to BU id of SQl bu table in final table 
                final_table["BU_id"].replace({final_table.iloc[num_BU,2]:insertObject[num_BU_Table]["id"]}, inplace=True)


except Exception as e: 
    print(e)
    InsertErrorMessageInSQL("Step6.1 Aqcure BU table & Convert BU_id",repr(e))
    
print("Step6.1 Complete!\n")


Step6.1 Complete!



In [22]:
final_table

,Fiscal_year,Fiscal_month,BU_id,Actual,Forecasts,Budgets
0,2022,Feb,1,171947.39,None,None
1,2022,Feb,2,170609.0,None,None
2,2022,Feb,3,50287.6,None,None
3,2022,Feb,4,97539.01,None,None
4,2022,Feb,5,-97571.44,None,None
...,...,...,...,...,...,...
8,2022,Dec,9,None,23014.0,148422.403923
9,2022,Dec,10,None,60000.0,57451.923538
10,2022,Dec,11,None,166666.0,166666.666333
11,2022,Dec,13,None,927945.0,None


#### Step6.2. Insert Value into EITDA Table in SQL Server

In [23]:
#Prepare the Stored procedure execution script and parameter 
storedProc = "Exec [dbo].[spEBITDA] @Fiscal_year =?, @Fiscal_month =?, @BU_id =?, @Actual =?, @Forecasts =?, @Budgets =?"

In [24]:
#Create List of final table to insert into SQL
final_list = final_table.values.tolist()

In [25]:
# insert data into SQL EBITDA table 
try:
    cursor.fast_executemany = True
    cursor.executemany(storedProc, final_list)
    # str(final_table.iat[-1,1] = month name of excel sheet (actual month name) 
    SucMSGList = ['Python', None, "Data Inserted Successfully for "+str(final_table.iat[-1,1]), 'No Error']
    
    # Insert data into SQL EBITDA table 
    cursor.execute("INSERT INTO translog (type, module_name, procedure_name, trans_details) VALUES (?,?,?,?)", SucMSGList)
    
    
except Exception as e:
    cursor.rollback()
    print(e.value)
    print('transaction rolled back')
    InsertErrorMessageInSQL("Step6.2. Insert Value into EITDA Table in SQL Server",repr(e))

else:
    print('records inserted successfully')
    cursor.commit()
    cursor.close() # data manipulation, keep the currsor active, otherwise close
finally:
    print('connection closed')
    conn.close() #Close Connection
    
print("Step6.2 Complete!\n")
print("All Done")
exit()

records inserted successfully
connection closed
Step6.2 Complete!

All Done
